# Task -1 :

## Create a custom data set and divide into batches

In [1]:
from sklearn.datasets import make_classification
import torch

In [3]:
# Step 1: Create a synthetic classification dataset using sklearn
X, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)

In [7]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [8]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [9]:
# Convert the array data to PyTorch tensors
X = torch.tensor(X,dtype=torch.float)
y = torch.tensor(y,dtype=torch.long)

In [10]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

# Dataset and Dataloader class

In [11]:
from torch.utils.data import Dataset, DataLoader

### Dataset class

In [12]:
# Creating a custom dataset class

class customdataset(Dataset): # Inherits the Dataset class

  def __init__(self,features,labels): # Constructor that defines how data should be loaded (Here features= x/inputs and labels=y/outputs)
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]  #Returns the total number of samples/rows in the dataset

  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [14]:
# Creating a object of the customdataset class
dataset = customdataset(X,y)

In [16]:
len(dataset)

10

In [23]:
# To access each data points/samples
dataset[0]  #has both features and labels

(tensor([ 1.0683, -0.9701]), tensor(1))

### Dataloader class

-> data loader class object is iterable , i.e we can loop through it to access batches

In [28]:
# No need to create a custom class fro dataloader , just create a object
# It takes 3 arguments (dataset_class_object , batch_size , whether_to_perform_suffle_or_not)
dataloader = DataLoader(dataset,batch_size=2,shuffle=True,num_workers=2) # Shuffle=true , randomly selects the order of index

# num_wrokers for parallelization of formation of batches/chunks

In [27]:
# To access the batch_features and batch_labels

for batch_features,batch_labels in dataloader:
  print(batch_features)
  print(batch_labels)
  print("-"*30)

# total number of batch is (total_datapoints / batch_size = 10 / 2 =5 )and  Each batch will have 2 data points(since batch_size=2)

tensor([[ 1.0683, -0.9701],
        [ 1.8997,  0.8344]])
tensor([1, 1])
------------------------------
tensor([[-1.9629, -0.9923],
        [-0.9382, -0.5430]])
tensor([0, 1])
------------------------------
tensor([[-2.8954,  1.9769],
        [-1.1402, -0.8388]])
tensor([0, 0])
------------------------------
tensor([[-0.5872, -1.9717],
        [ 1.7774,  1.5116]])
tensor([0, 1])
------------------------------
tensor([[ 1.7273, -1.1858],
        [-0.7206, -0.9606]])
tensor([1, 0])
------------------------------


# Task - 2 : Implement dataset and dataloader class in existing brest cancer dataset from previous lessons

In [47]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn


In [30]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [31]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [33]:
# Perform train test split on the dataset
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [34]:
# Standard scaler on input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [35]:
# Label encoder on output features
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [36]:
# Convert the numpy arrays into pytorch tensors
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [43]:
len(X_train_tensor)

455

# Dataset class

#### Creating a customdataset class with 3 key fxn defined

In [37]:
class customdataset(Dataset):

  def __init__(self,features,labels):   # Defines how to access and load the data
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]   # function that returns the total number of samples/rows in the dataste

  def __getitem__(self,index):
    return self.features[index],self.labels[index]  #Function that returns the features and lables for each index input index value

In [38]:
# Creating 2 objects of dataset class , one for training data and one for test data

train_dataset = customdataset(X_train_tensor,y_train_tensor)   # train_dataset have all the training data (features and labels)
test_dataset = customdataset(X_test_tensor,y_test_tensor)

In [41]:
len(train_dataset)

455

In [42]:
len(test_dataset)

114

In [44]:
train_dataset[0]

(tensor([ 1.0517, -2.0873,  1.2135,  0.9316,  1.5782,  3.2015,  2.5957,  2.4353,
          2.2136,  2.3090,  2.3747, -0.5858,  2.6692,  2.3177, -0.2114,  1.3257,
          0.8030,  0.6701,  1.1553,  0.9498,  1.8233, -1.3538,  2.2128,  1.9192,
          1.3576,  2.6041,  2.0843,  2.2531,  2.8659,  2.0018]),
 tensor(1.))

# Dataloader class

In [45]:
train_loader = DataLoader(train_dataset,batch_size=32 , shuffle=True,num_workers=2)
test_loader = DataLoader(test_dataset,batch_size=32 , shuffle=True,num_workers=2)

# Define the model

In [60]:
class networkmodel(nn.Module):  # Must inherit the nn.Module class inorder to use "nn module"

  def __init__(self,num_features):

    super().__init__()   #----> invoking the parent class(nn.module) constructor using super keyword

     # Creating a Sequential container for the neural n/w
    self.network = nn.Sequential(
        nn.Linear(num_features,30),   # (hidden layer) Accepts 2 paramaters , (numer_of_input_features , number_of_outputs)
        nn.ReLU(),
        nn.Linear(30,1),
        nn.Sigmoid()
    )

  def forward(self,features):  # Function for forward propogation
    return self.network(features)




# Training pipeline

In [52]:
# Define the loss function
loss_function = nn.BCELoss()

In [63]:
# Import paramaters:
learning_rate = 0.1
epochs = 10

In [64]:
# Creating a model object
model = networkmodel(X_train_tensor.shape[1])

# Define the optimizer

optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

# define outer loop for epoch iteration
for epoch in range(epochs):

  # define inner loop to iterate over the train data loader , to access and pass the batch_features and batch_labels for each batch
  for batch_features , batch_labels in train_loader:

    # forward pass
    y_pred = model.forward(batch_features)

    #Loss calculate
    loss = loss_function(y_pred , batch_labels.reshape(-1,1))

    # clear gradients
    optimizer.zero_grad()

    #backward pass / back propogation
    loss.backward()

    # update gradients
    optimizer.step()

    print(f'Epoch:{epoch+1}, loss: {loss.item()}')



Epoch:1, loss: 0.6133289337158203
Epoch:1, loss: 0.5482474565505981
Epoch:1, loss: 0.5592692494392395
Epoch:1, loss: 0.5260929465293884
Epoch:1, loss: 0.4715842008590698
Epoch:1, loss: 0.4952777624130249
Epoch:1, loss: 0.45715945959091187
Epoch:1, loss: 0.41361749172210693
Epoch:1, loss: 0.4076111912727356
Epoch:1, loss: 0.4544302225112915
Epoch:1, loss: 0.3486791253089905
Epoch:1, loss: 0.3049308657646179
Epoch:1, loss: 0.32336246967315674
Epoch:1, loss: 0.30613723397254944
Epoch:1, loss: 0.3322587013244629
Epoch:2, loss: 0.25715142488479614
Epoch:2, loss: 0.2975161671638489
Epoch:2, loss: 0.30338138341903687
Epoch:2, loss: 0.2538467049598694
Epoch:2, loss: 0.32458510994911194
Epoch:2, loss: 0.25412803888320923
Epoch:2, loss: 0.21338355541229248
Epoch:2, loss: 0.2780938744544983
Epoch:2, loss: 0.24925756454467773
Epoch:2, loss: 0.22122666239738464
Epoch:2, loss: 0.33935609459877014
Epoch:2, loss: 0.18986845016479492
Epoch:2, loss: 0.2453477382659912
Epoch:2, loss: 0.17529286444187164


# Evaluation

In [67]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.5).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9783
